In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
data = pd.read_csv("emotions.csv")

# Data preprocessing
X = data['text']
y = data['label']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding sequences
max_length = 100  # Define your maximum sequence length
X = pad_sequences(X, maxlen=max_length)

# Convert labels to one-hot encoding
num_classes = len(data['label'].unique())
y = pd.get_dummies(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Predict labels
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(np.array(y_test), axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)


Epoch 1/10
8337/8337 [==============================] - 1491s 178ms/step - loss: 0.2161 - accuracy: 0.9039 - val_loss: 0.1005 - val_accuracy: 0.9354
Epoch 2/10
8337/8337 [==============================] - 1458s 175ms/step - loss: 0.0989 - accuracy: 0.9387 - val_loss: 0.0944 - val_accuracy: 0.9358
Epoch 3/10
8337/8337 [==============================] - 1461s 175ms/step - loss: 0.0886 - accuracy: 0.9421 - val_loss: 0.0941 - val_accuracy: 0.9341
Epoch 4/10
8337/8337 [==============================] - 1430s 171ms/step - loss: 0.0845 - accuracy: 0.9443 - val_loss: 0.0954 - val_accuracy: 0.9295
Epoch 5/10
8337/8337 [==============================] - 1397s 168ms/step - loss: 0.0823 - accuracy: 0.9465 - val_loss: 0.0987 - val_accuracy: 0.9289
Epoch 6/10
2606/2606 [==============================] - 71s 27ms/step - loss: 0.0940 - accuracy: 0.9353
Test Loss: 0.09398199617862701
Test Accuracy: 0.9352942705154419
2606/2606 [==============================] - 70s 27ms/step
Precision: 0.93475035502570